In [1]:
import os
import sys
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Embedding, LSTM
from sklearn.utils import shuffle
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from stop_words import get_stop_words
from pandas import Series, DataFrame

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
df = pd.read_csv('../data/train_baseline.csv')
df = shuffle(df)

In [7]:
data =  np.array(df['data'])
y = np.array(df['label'])

y = pd.get_dummies(y)
x = []
for i in data:
    i = i.strip('[|]')
    x.append(i.split(','))
x = np.array(x)
print(len(data))

10000


In [8]:
train_x = x[0: 5000]
train_y = y[0: 5000]
print(sum(train_y))
print(len(train_y))
test_x = x[5000:]
test_y = y[5000:]
print(sum(test_y))
print(len(test_y))

1
5000
1
5000


In [9]:
#MLP model
model = Sequential()
model.add(Dense(64, input_dim=train_x.shape[1], activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [6]:
model.fit(train_x, train_y, epochs=50, batch_size=128)

Epoch 1/50
5000/5000 [==============================] - 1s 181us/step - loss: 7.8824 - acc: 0.5076
Epoch 2/50
5000/5000 [==============================] - 1s 102us/step - loss: 8.2261 - acc: 0.4882
Epoch 3/50
5000/5000 [==============================] - 1s 102us/step - loss: 8.1767 - acc: 0.4920
Epoch 4/50
5000/5000 [==============================] - 1s 101us/step - loss: 8.0603 - acc: 0.4996
Epoch 5/50
5000/5000 [==============================] - 1s 114us/step - loss: 8.2358 - acc: 0.4878
Epoch 6/50
5000/5000 [==============================] - 1s 105us/step - loss: 8.2269 - acc: 0.4876
Epoch 7/50
5000/5000 [==============================] - 1s 103us/step - loss: 8.2645 - acc: 0.4844
Epoch 8/50
5000/5000 [==============================] - 1s 105us/step - loss: 8.2800 - acc: 0.4830
Epoch 9/50
5000/5000 [==============================] - 1s 120us/step - loss: 8.3454 - acc: 0.4804
Epoch 10/50
5000/5000 [==============================] - 1s 104us/step - loss: 8.2371 - acc: 0.4870
Epoch 11/

In [8]:
#evaluate the model
model.evaluate(test_x, test_y)

7072/7072 [==============================] - 1s 138us/step


[5.934271267636329, 0.6298076923076923]

In [17]:
predict_x = model.predict_proba(test_x)
print(len(predict_x))
print(predict_x)

7072
[[1.]
 [1.]
 [1.]
 ...
 [1.]
 [1.]
 [1.]]


In [9]:
vocabulary = pd.read_csv('../data/vocabulary.csv')
train_question = pd.read_csv('../data/FiQA_train_question_final.tsv', sep='\t')
train_doc = pd.read_csv('../data/FiQA_train_doc_final.tsv', sep='\t')
train_doc = train_doc.dropna()

dictionary = dict(zip(list(vocabulary['token']), list(vocabulary['word'])))
vocab_size = len(dictionary)
VOCAB_PAD_ID = vocab_size + 1
VOCAB_GO_ID = vocab_size + 2

In [10]:
def padding(lis, pad, size):
    if size > len(lis):
        lis += [pad] * (size - len(lis))
    else:
        lis = lis[0:size]
    return lis
    
def text_to_id(texts, dictionary):
    #get the dictionary of the vocalbulary in the corpus, the word_idct is the word and id mapping in the dictionary
    # change the question and docs to id list
    texts_id = []
    for text in texts:
        t = []
        for word in text:
            if word in dictionary:
                t.append(dictionary[word])
#             else:
#                 print(word)
        texts_id.append(t)
    return texts_id

def create_input(questions, docs, dictionary, isPositive = 1):
    questions_id = text_to_id(questions, dictionary)
    docs_id =  text_to_id(docs, dictionary)
    # add the label to the corresponding question and docs
    q_len = len(questions)
    input_data = []
    for i in range(0, q_len):
        item = {}
        item['question'] = questions_id[i]
        item['doc'] = docs_id[i]
        if isPositive:
            item['label'] = [0, 1]
        else:
            item['label'] = [1, 0]
        input_data.append(item)
    return input_data

def pack_question_n_utterance(q, doc, VOCAB_PAD_ID, VOCAB_GO_ID, q_length = 20, doc_length = 99):
    q = padding(q, VOCAB_PAD_ID, q_length)
    doc = padding(doc, VOCAB_PAD_ID, doc_length)
    assert len(q) == q_length, "question should be pad to q_length"
    assert len(doc) == doc_length, "doc should be pad to doc_length"
    return q + [VOCAB_GO_ID] + doc

def preprocess_data(data, VOCAB_PAD_ID, VOCAB_GO_ID, question_max_length = 20, doc_max_length = 99):
    result_x = []
    result_y = []
    for o in data:
        x = pack_question_n_utterance(o['question'], o['doc'], VOCAB_PAD_ID, VOCAB_GO_ID)
        y_ = o['label']
        assert len(x) == doc_max_length + question_max_length + 1, "Wrong length afer padding"
        assert VOCAB_GO_ID in x, "<GO> must be in input x"
        result_x.append(x)
        result_y.append(y_)
    
    return [result_x, result_y]
def splitWordByLibrary(documents):
    texts = []
    tokenizer = RegexpTokenizer(r'\w+')
    en_stop = get_stop_words('en')
    p_stemmer = PorterStemmer()

    for i in documents:
        # clean and tokenize document string
        raw = i.lower()
        tokens = tokenizer.tokenize(raw)

        # remove stop words from tokens
        stopped_tokens = [i for i in tokens if not i in en_stop]

        # stem tokens
        stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]

        # add tokens to list
        texts.append(stemmed_tokens)
        
    return texts

In [12]:
docs = train_doc['doc'].values
docs = splitWordByLibrary(docs) 

questions = train_question['question'].sample(1).values
questions = splitWordByLibrary(questions)
questions = questions*len(docs)

In [13]:
input_data = create_input(questions, docs, dictionary)

In [14]:
print(len(input_data))

57600


In [15]:
x, y = preprocess_data(input_data, VOCAB_PAD_ID, VOCAB_GO_ID)

In [16]:
x = np.array(x)
print(x.shape)

(57600, 120)


In [24]:
y_ = model.predict(x)

In [25]:
print(type(train_doc['docid'].values))
print(y_)

<class 'numpy.ndarray'>
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]


In [26]:
df = DataFrame(data = {'docid': list(train_doc['docid'].values), 'y': list(y_)})
print(df)

        docid           y
0           3  [0.0, 1.0]
1          31  [0.0, 1.0]
2          56  [0.0, 1.0]
3          59  [0.0, 1.0]
4          63  [0.0, 1.0]
5         100  [0.0, 1.0]
6         108  [0.0, 1.0]
7         125  [0.0, 1.0]
8         132  [0.0, 1.0]
9         138  [0.0, 1.0]
10        158  [0.0, 1.0]
11        164  [0.0, 1.0]
12        178  [0.0, 1.0]
13        180  [0.0, 1.0]
14        222  [0.0, 1.0]
15        225  [0.0, 1.0]
16        239  [0.0, 1.0]
17        264  [0.0, 1.0]
18        277  [0.0, 1.0]
19        288  [0.0, 1.0]
20        289  [0.0, 1.0]
21        294  [0.0, 1.0]
22        297  [0.0, 1.0]
23        302  [0.0, 1.0]
24        316  [0.0, 1.0]
25        330  [0.0, 1.0]
26        339  [0.0, 1.0]
27        343  [0.0, 1.0]
28        357  [0.0, 1.0]
29        396  [0.0, 1.0]
...       ...         ...
57570  599651  [0.0, 1.0]
57571  599678  [0.0, 1.0]
57572  599684  [0.0, 1.0]
57573  599700  [0.0, 1.0]
57574  599701  [0.0, 1.0]
57575  599715  [0.0, 1.0]
57576  59972